<a href="https://colab.research.google.com/github/bazuhiroki/web-/blob/main/%E8%B3%87%E6%A0%BC%E5%8F%96%E5%BE%97%E7%AE%A1%E7%90%86%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import { useState, useEffect, useCallback } from 'react';
import * as Plotly from 'plotly.js/dist/plotly';
import { v4 as uuidv4 } from 'uuid';
import { GoogleGenerativeAI } from '@google/generative-ai';

const App = () => {
    const genAI = new GoogleGenerativeAI("");

    // State variables for the app
    const [data, setData] = useState({
        certifications: [],
        progress: {},
    });
    const [currentTab, setCurrentTab] = useState('tab-1');
    const [editingCert, setEditingCert] = useState(null);
    const [currentCertId, setCurrentCertId] = useState(null);
    const [certForm, setCertForm] = useState({
        name: '',
        difficulty: 1,
        necessity: 1,
        gai: 1,
    });
    const [loading, setLoading] = useState(false);
    const [report, setReport] = useState('');
    const [reporter, setReporter] = useState('');

    // Handle Excel file upload for certifications
    const handleCertFileUpload = (e) => {
        const file = e.target.files[0];
        if (!file) return;

        const reader = new FileReader();
        reader.onload = (event) => {
            const result = event.target.result;
            // `exceljs` を使用せず、JSON形式のデータを想定した簡易的なデータ読み込み
            try {
                const parsedData = JSON.parse(result);
                if (Array.isArray(parsedData) && parsedData.length > 0) {
                    const certifications = parsedData.map(cert => ({
                        id: uuidv4(),
                        name: cert.name,
                        difficulty: cert.difficulty,
                        necessity: cert.necessity,
                        gai: cert.gai,
                    }));
                    setData(prevData => ({ ...prevData, certifications }));
                } else {
                    alert('無効なファイル形式です。JSON形式のデータをアップロードしてください。');
                }
            } catch (error) {
                alert('ファイルの読み込み中にエラーが発生しました。JSON形式のデータか確認してください。');
            }
        };
        reader.readAsText(file);
    };

    // Load initial data from local storage on first load
    useEffect(() => {
        const storedData = JSON.parse(localStorage.getItem('shikakuData'));
        if (storedData) {
            setData(storedData);
        }
    }, []);

    // Save data to local storage whenever it changes
    useEffect(() => {
        localStorage.setItem('shikakuData', JSON.stringify(data));
    }, [data]);

    // Update form when editing a certification
    useEffect(() => {
        if (editingCert) {
            setCertForm({
                name: editingCert.name,
                difficulty: editingCert.difficulty,
                necessity: editingCert.necessity,
                gai: editingCert.gai,
            });
        } else {
            setCertForm({
                name: '',
                difficulty: 1,
                necessity: 1,
                gai: 1,
            });
        }
    }, [editingCert]);

    // Handle form input changes
    const handleFormChange = (e) => {
        const { name, value } = e.target;
        setCertForm((prevForm) => ({ ...prevForm, [name]: value }));
    };

    // Handle add/update certification
    const handleAddUpdateCert = () => {
        const newCert = {
            id: editingCert ? editingCert.id : uuidv4(),
            ...certForm,
        };

        if (editingCert) {
            setData((prevData) => ({
                ...prevData,
                certifications: prevData.certifications.map((c) =>
                    c.id === newCert.id ? newCert : c
                ),
            }));
            setEditingCert(null);
        } else {
            setData((prevData) => ({
                ...prevData,
                certifications: [...prevData.certifications, newCert],
            }));
        }
        setCertForm({ name: '', difficulty: 1, necessity: 1, gai: 1 });
    };

    // Handle delete certification
    const handleDeleteCert = (id) => {
        if (window.confirm('この資格情報を削除してもよろしいですか？')) {
            setData((prevData) => ({
                ...prevData,
                certifications: prevData.certifications.filter((c) => c.id !== id),
            }));
        }
    };

    // Render the 3D scatter plot
    const renderScatterPlot = useCallback(() => {
        const df = data.certifications;
        if (df.length === 0) return;

        const plotData = [
            {
                x: df.map((c) => c.difficulty),
                y: df.map((c) => c.necessity),
                z: df.map((c) => c.gai),
                mode: 'markers',
                type: 'scatter3d',
                marker: {
                    size: df.map((c) => c.gai),
                    color: df.map((c) => c.gai),
                    colorscale: 'Cividis',
                    colorbar: { title: 'やりがい' },
                },
                text: df.map((c) => c.name),
                hoverinfo: 'text',
            },
        ];

        const layout = {
            height: '100%',
            paper_bgcolor: 'transparent',
            plot_bgcolor: 'transparent',
            margin: { l: 0, r: 0, b: 0, t: 0 },
            scene: {
                xaxis: { title: '難易度', range: [0, 6], showgrid: true, showbackground: false },
                yaxis: { title: '必要性', range: [0, 6], showgrid: true, showbackground: false },
                zaxis: { title: 'やりがい', range: [0, 11], showgrid: true, showbackground: false },
                aspectratio: { x: 1, y: 1, z: 1 },
            },
        };

        Plotly.newPlot('scatter-plot', plotData, layout, { responsive: true });
    }, [data.certifications]);

    useEffect(() => {
        if (document.getElementById('scatter-plot')) {
            renderScatterPlot();
        }
    }, [renderScatterPlot]);

    // Handle progress file upload
    const handleProgressFileUpload = (e) => {
        const file = e.target.files[0];
        if (!file || !currentCertId) return;

        const reader = new FileReader();
        reader.onload = (event) => {
            const result = event.target.result;
            try {
                const workbook = new ExcelJS.Workbook();
                workbook.xlsx.load(result).then((wb) => {
                    const worksheet = wb.getWorksheet(1);
                    const items = {};
                    worksheet.eachRow({ includeEmpty: false }, (row) => {
                        const cellValue = row.getCell(1).value;
                        if (typeof cellValue === 'string') {
                            const trimmedValue = cellValue.trim();
                            if (trimmedValue.startsWith('●') || trimmedValue.startsWith('・')) {
                                items[uuidv4()] = {
                                    name: trimmedValue,
                                    checked: false,
                                    type: trimmedValue.startsWith('●') ? 'major' : 'minor',
                                };
                            }
                        }
                    });

                    setData((prevData) => ({
                        ...prevData,
                        progress: {
                            ...prevData.progress,
                            [currentCertId]: {
                                ...prevData.progress[currentCertId],
                                items,
                            },
                        },
                    }));
                });
            } catch (error) {
                alert('ファイルの読み込み中にエラーが発生しました。Excel形式のファイルか確認してください。');
            }
        };
        reader.readAsArrayBuffer(file);
    };

    // Handle checkbox change for progress items
    const handleProgressCheckbox = (itemId) => {
        setData((prevData) => {
            const updatedProgress = { ...prevData.progress };
            const currentCert = updatedProgress[currentCertId];
            if (currentCert && currentCert.items) {
                currentCert.items = {
                    ...currentCert.items,
                    [itemId]: {
                        ...currentCert.items[itemId],
                        checked: !currentCert.items[itemId].checked,
                    },
                };
            }
            return {
                ...prevData,
                progress: updatedProgress,
            };
        });
    };

    // Generate AI Report
    const generateReport = async () => {
        setLoading(true);
        try {
            const certInfo = data.certifications.find(c => c.id === currentCertId);
            const progressData = data.progress[currentCertId];

            if (!certInfo || !progressData || !progressData.items) {
                setReport('進捗データが不完全です。');
                setLoading(false);
                return;
            }

            const totalMinorItems = Object.values(progressData.items).filter(i => i.type === 'minor').length;
            const checkedMinorItems = Object.values(progressData.items).filter(i => i.type === 'minor' && i.checked).length;
            const progressPercent = totalMinorItems > 0 ? (checkedMinorItems / totalMinorItems) * 100 : 0;
            const certName = certInfo.name;

            const prompt = `
            以下の学習進捗状況について、${reporter}の特徴になりきって、進捗評価とモチベーションを高めるためのアドバイスをしてください。
            進捗状況：
            - 学習中の資格：${certName}
            - 全学習項目数：${totalMinorItems}
            - 完了した項目数：${checkedMinorItems}
            - 進捗率：${progressPercent.toFixed(1)}%
            `;

            const model = genAI.getGenerativeModel({ model: "gemini-pro" });
            const result = await model.generateContent(prompt);
            const response = await result.response;
            const text = response.text;
            setReport(text);
        } catch (error) {
            setReport(`レポート生成中にエラーが発生しました: ${error.message}`);
        }
        setLoading(false);
    };

    // Render Tab 1: Priority Dashboard
    const renderTab1 = () => {
        const df = data.certifications;
        return (
            <div className="flex flex-col lg:flex-row p-4 space-y-4 lg:space-y-0 lg:space-x-4">
                <div className="w-full lg:w-1/3 space-y-4">
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">Excelから資格情報をアップロード</h3>
                        <p className="text-sm text-gray-400 mb-2">
                            6行目をヘッダーとして読み込みます。列名は「項目名/軸名」「難易度」「必要性」「やりがい」が必要です。
                        </p>
                        <input
                            type="file"
                            onChange={handleCertFileUpload}
                            className="block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-violet-50 file:text-violet-700 hover:file:bg-violet-100"
                        />
                    </div>
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">資格情報の追加/編集</h3>
                        <div className="mb-4">
                            <label className="block text-sm font-medium mb-1">資格名</label>
                            <input
                                type="text"
                                name="name"
                                value={certForm.name}
                                onChange={handleFormChange}
                                placeholder="例：基本情報技術者試験"
                                className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                            />
                        </div>
                        <div className="mb-4">
                            <label className="block text-sm font-medium mb-1">難易度 (1-5)</label>
                            <input
                                type="number"
                                name="difficulty"
                                value={certForm.difficulty}
                                onChange={handleFormChange}
                                min="1"
                                max="5"
                                step="1"
                                className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                            />
                        </div>
                        <div className="mb-4">
                            <label className="block text-sm font-medium mb-1">必要性 (1-5)</label>
                            <input
                                type="number"
                                name="necessity"
                                value={certForm.necessity}
                                onChange={handleFormChange}
                                min="1"
                                max="5"
                                step="1"
                                className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                            />
                        </div>
                        <div className="mb-4">
                            <label className="block text-sm font-medium mb-1">やりがい (1-10)</label>
                            <input
                                type="number"
                                name="gai"
                                value={certForm.gai}
                                onChange={handleFormChange}
                                min="1"
                                max="10"
                                step="0.1"
                                className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                            />
                        </div>
                        <button
                            onClick={handleAddUpdateCert}
                            className="w-full bg-blue-600 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded-lg transition-colors duration-200"
                        >
                            {editingCert ? '更新' : '追加'}
                        </button>
                        {editingCert && (
                            <button
                                onClick={() => setEditingCert(null)}
                                className="w-full bg-gray-500 hover:bg-gray-600 text-white font-bold py-2 px-4 rounded-lg mt-2 transition-colors duration-200"
                            >
                            キャンセル
                            </button>
                        )}
                    </div>
                </div>

                <div className="w-full lg:w-2/3 space-y-4">
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">資格取得の優先順位（3D散布図）</h3>
                        <div id="scatter-plot" className="w-full h-[60vh]"></div>
                    </div>
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">資格一覧</h3>
                        <div className="overflow-x-auto">
                            <table className="min-w-full divide-y divide-gray-700">
                                <thead className="bg-gray-700">
                                    <tr>
                                        <th scope="col" className="px-6 py-3 text-left text-xs font-medium text-gray-300 uppercase tracking-wider">資格名</th>
                                        <th scope="col" className="px-6 py-3 text-left text-xs font-medium text-gray-300 uppercase tracking-wider">難易度</th>
                                        <th scope="col" className="px-6 py-3 text-left text-xs font-medium text-gray-300 uppercase tracking-wider">必要性</th>
                                        <th scope="col" className="px-6 py-3 text-left text-xs font-medium text-gray-300 uppercase tracking-wider">やりがい</th>
                                        <th scope="col" className="px-6 py-3 text-left text-xs font-medium text-gray-300 uppercase tracking-wider">アクション</th>
                                    </tr>
                                </thead>
                                <tbody className="bg-gray-800 divide-y divide-gray-700">
                                    {df.length === 0 ? (
                                        <tr>
                                            <td colSpan="5" className="px-6 py-4 whitespace-nowrap text-center text-sm font-medium text-gray-400">
                                                資格情報が登録されていません。
                                            </td>
                                        </tr>
                                    ) : (
                                        df.map((cert) => (
                                            <tr key={cert.id} className="hover:bg-gray-700 transition-colors duration-150">
                                                <td className="px-6 py-4 whitespace-nowrap text-sm font-medium text-white">{cert.name}</td>
                                                <td className="px-6 py-4 whitespace-nowrap text-sm text-gray-300">{cert.difficulty}</td>
                                                <td className="px-6 py-4 whitespace-nowrap text-sm text-gray-300">{cert.necessity}</td>
                                                <td className="px-6 py-4 whitespace-nowrap text-sm text-gray-300">{cert.gai}</td>
                                                <td className="px-6 py-4 whitespace-nowrap text-right text-sm font-medium">
                                                    <button onClick={() => setEditingCert(cert)} className="text-indigo-400 hover:text-indigo-600 mr-4">編集</button>
                                                    <button onClick={() => handleDeleteCert(cert.id)} className="text-red-400 hover:text-red-600">削除</button>
                                                </td>
                                            </tr>
                                        ))
                                    )}
                                </tbody>
                            </table>
                        </div>
                    </div>
                </div>
            </div>
        );
    };

    // Render Tab 2: Progress Management
    const renderTab2 = () => {
        const certs = data.certifications;
        const currentCertProgress = data.progress[currentCertId] || {};
        const totalMinorItems = Object.values(currentCertProgress.items || {}).filter(i => i.type === 'minor').length;
        const checkedMinorItems = Object.values(currentCertProgress.items || {}).filter(i => i.type === 'minor' && i.checked).length;
        const progressPercent = totalMinorItems > 0 ? (checkedMinorItems / totalMinorItems) * 100 : 0;

        return (
            <div className="flex flex-col lg:flex-row p-4 space-y-4 lg:space-y-0 lg:space-x-4">
                <div className="w-full lg:w-1/3 space-y-4">
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">学習中の資格設定</h3>
                        <div className="mb-4">
                            <label className="block text-sm font-medium mb-1">学習中の資格</label>
                            <select
                                className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                                value={currentCertId || ''}
                                onChange={(e) => setCurrentCertId(e.target.value)}
                            >
                                <option value="">資格を選択...</option>
                                {certs.map((cert) => (
                                    <option key={cert.id} value={cert.id}>
                                        {cert.name}
                                    </option>
                                ))}
                            </select>
                        </div>
                        <div className="mb-4">
                            <label className="block text-sm font-medium mb-1">目標取得日</label>
                            <input
                                type="date"
                                value={currentCertProgress.targetDate || ''}
                                onChange={(e) => {
                                    setData(prevData => ({
                                        ...prevData,
                                        progress: {
                                            ...prevData.progress,
                                            [currentCertId]: {
                                                ...prevData.progress[currentCertId],
                                                targetDate: e.target.value,
                                            },
                                        },
                                    }));
                                }}
                                className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                            />
                        </div>
                    </div>
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">学習項目アップロード</h3>
                        <p className="text-sm text-gray-400 mb-2">大項目・中項目を定義したExcelファイルをアップロードします。</p>
                        <input type="file" onChange={handleProgressFileUpload} className="block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-violet-50 file:text-violet-700 hover:file:bg-violet-100"/>
                        <button
                            onClick={() => {
                                const newProgress = { ...currentCertProgress, items: {} };
                                setData(prevData => ({
                                    ...prevData,
                                    progress: { ...prevData.progress, [currentCertId]: newProgress }
                                }));
                            }}
                            className="w-full bg-red-600 hover:bg-red-700 text-white font-bold py-2 px-4 rounded-lg mt-4 transition-colors duration-200"
                        >
                            進捗状況をリセット
                        </button>
                    </div>
                </div>

                <div className="w-full lg:w-2/3 space-y-4">
                    <div className="bg-gray-800 p-4 rounded-lg shadow-lg">
                        <h3 className="text-xl font-bold mb-4">{data.certifications.find(c => c.id === currentCertId)?.name || '資格を選択してください'}</h3>
                        <div className="w-full bg-gray-700 rounded-full h-4 mb-4">
                            <div className="bg-green-500 h-4 rounded-full text-xs flex items-center justify-center text-white" style={{ width: `${progressPercent}%` }}>
                                {progressPercent.toFixed(1)}%
                            </div>
                        </div>
                        <h4 className="text-lg font-bold mb-2">学習項目一覧</h4>
                        <div className="space-y-2 max-h-[40vh] overflow-y-auto pr-2">
                            {Object.entries(currentCertProgress.items || {}).map(([key, item]) => (
                                <div key={key}>
                                    {item.type === 'major' ? (
                                        <h5 className="font-bold mt-4 mb-1">{item.name.replace('●', '').trim()}</h5>
                                    ) : (
                                        <label className="flex items-center space-x-2">
                                            <input type="checkbox" checked={item.checked} onChange={() => handleProgressCheckbox(key)} className="form-checkbox h-4 w-4 text-blue-600 bg-gray-900 border-gray-700 rounded focus:ring-blue-500" />
                                            <span className="text-gray-300">{item.name.replace('・', '').trim()}</span>
                                        </label>
                                    )}
                                </div>
                            ))}
                        </div>
                    </div>
                </div>
            </div>
        );
    };

    // Render Tab 3: AI Report
    const renderTab3 = () => {
        return (
            <div className="flex flex-col p-4 space-y-4">
                <div className="bg-gray-800 p-4 rounded-lg shadow-lg w-full">
                    <h3 className="text-xl font-bold mb-4">AI進捗レポート</h3>
                    <p className="text-sm text-gray-400 mb-4">進捗状況に基づいて、AIがあなたにアドバイスを送ります。</p>
                    <div className="mb-4">
                        <label className="block text-sm font-medium mb-1">レポート者（著名人）</label>
                        <input
                            type="text"
                            value={reporter}
                            onChange={(e) => setReporter(e.target.value)}
                            placeholder="例：スティーブ・ジョブズ"
                            className="w-full p-2 bg-gray-700 rounded-md border border-gray-600 focus:outline-none focus:ring-2 focus:ring-blue-500"
                        />
                    </div>
                    <button
                        onClick={generateReport}
                        className="w-full bg-green-600 hover:bg-green-700 text-white font-bold py-2 px-4 rounded-lg transition-colors duration-200"
                        disabled={loading || !currentCertId}
                    >
                        {loading ? '生成中...' : 'レポートを生成'}
                    </button>
                    {report && (
                        <div className="mt-4 p-4 bg-gray-700 rounded-lg whitespace-pre-wrap">
                            {report}
                        </div>
                    )}
                </div>
            </div>
        );
    };

    const renderContent = () => {
        switch (currentTab) {
            case 'tab-1':
                return renderTab1();
            case 'tab-2':
                return renderTab2();
            case 'tab-3':
                return renderTab3();
            default:
                return null;
        }
    };

    return (
        <div className="min-h-screen bg-gray-900 text-gray-100 font-sans p-4">
            <header className="text-center mb-8">
                <h1 className="text-4xl font-extrabold text-white">資格取得管理ダッシュボード</h1>
            </header>

            <nav className="mb-8">
                <div className="border-b border-gray-700">
                    <div className="flex justify-center space-x-4">
                        <button
                            className={`py-2 px-4 rounded-t-lg transition-colors duration-200 ${currentTab === 'tab-1' ? 'bg-gray-800 text-white' : 'text-gray-400 hover:bg-gray-700'}`}
                            onClick={() => setCurrentTab('tab-1')}
                        >
                            優先順位付け
                        </button>
                        <button
                            className={`py-2 px-4 rounded-t-lg transition-colors duration-200 ${currentTab === 'tab-2' ? 'bg-gray-800 text-white' : 'text-gray-400 hover:bg-gray-700'}`}
                            onClick={() => setCurrentTab('tab-2')}
                        >
                            進捗管理
                        </button>
                        <button
                            className={`py-2 px-4 rounded-t-lg transition-colors duration-200 ${currentTab === 'tab-3' ? 'bg-gray-800 text-white' : 'text-gray-400 hover:bg-gray-700'}`}
                            onClick={() => setCurrentTab('tab-3')}
                        >
                            レポート
                        </button>
                    </div>
                </div>
            </nav>

            <main className="container mx-auto">
                {renderContent()}
            </main>
        </div>
    );
};

export default App;


SyntaxError: invalid character '、' (U+3001) (ipython-input-727290170.py, line 35)